In [1]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import numpy as np

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_pickle('data_raw.pickle')
df.head(3)

,tconst,primaryTitle,averageRating,vote_count,vote_average,revenue,budget,num_genres,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,History,Horror,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western,inconnu,ratio_votes,originalTitle,startYear,genres_x,runtime,overview,original_language,poster_path,language_fact,Director,cast,actor_1,actor_2,actor_3,actor_4,actor_1_fact,actor_2_fact,actor_3_fact,actor_4_fact,director_fact,genres_fact
0,tt0499549,Avatar,7.9,29045,7.600,2923706026,237000000,3,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3676.582278,Avatar,2009,"Action,Adventure,Fantasy",162,"In the 22nd century, a paraplegic Marine is di...",anglais,/jRXYjXNq0Cs2TcJjLkki24MLp7u.jpg,0,James Cameron,"Sigourney Weaver, Michelle Rodriguez, Sam Wort...",Sigourney Weaver,Michelle Rodriguez,Sam Worthington,Zoe Saldana,0,0,0,0,0,0
1,tt4154796,Avengers: Endgame,8.4,23091,8.267,2794731755,356000000,3,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2748.928571,Avengers: Endgame,2019,"Action,Adventure,Drama",181,After the devastating events of Avengers: Infi...,anglais,/or06FN3Dka5tukK1e9sl16pB3iy.jpg,0,Anthony Russo,"Robert Downey Jr., Mark Ruffalo, Chris Evans, ...",Robert Downey Jr.,Mark Ruffalo,Chris Evans,Chris Hemsworth,1,1,1,1,1,1
3,tt1630029,Avatar: The Way of Water,7.6,7846,7.707,2319331580,460000000,3,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1032.368421,Avatar: The Way of Water,2022,"Action,Adventure,Fantasy",192,Set more than a decade after the events of the...,anglais,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg,0,James Cameron,"Sigourney Weaver, Stephen Lang, Sam Worthingto...",Sigourney Weaver,Stephen Lang,Sam Worthington,Zoe Saldana,0,2,0,0,0,0


In [6]:
df.loc[0,'poster_path']

'/jRXYjXNq0Cs2TcJjLkki24MLp7u.jpg'

In [3]:
table_totale_brute = pd.read_pickle('data_raw.pickle')

In [4]:
def filtrer_table(table_totale_brute):
    # -------------------------------------------------------------------------------------------------------------- GENRE
    # Convertir et formater la colonne 'genres_x'
    table_totale_brute['genres_x'] = table_totale_brute['genres_x'].astype(str).str.lower()
    # Créer une copie du DataFrame pour éviter de modifier l'original
    genre = table_totale_brute.copy()
    # Diviser la colonne 'genres_x' en listes
    genre['genres_x'] = genre['genres_x'].apply(lambda x: x.split(','))
    # Explosion du DataFrame pour traiter les listes de genres
    genre_1 = genre.explode('genres_x')
    # Remplacer les valeurs NaN dans la colonne 'genres_x' par 'inconnu'
    genre_1['genres_x'].replace('nan', 'inconnu', inplace= True)
    # Obtenir la liste unique des genres
    list_genres = genre_1['genres_x'].unique()
    # Saisie manuelle du genre par l'utilisateur
    genre_choisi = input(f"Choisissez un genre parmi {list_genres} (laissez vide pour tous): ")
    # Filtrer le DataFrame en fonction du genre choisi
    if genre_choisi:
        table_totale_brute = genre_1[genre_1['genres_x'] == genre_choisi]
    # -------------------------------------------------------------------------------------------------------------- ANNÉE
    # Saisie manuelle de l'année par l'utilisateur
    annee_choisie = input("Entrez une année (laissez vide pour tous) : ")
    if annee_choisie:
        # Convertir la colonne 'release_date' en datetime
        # table_totale_brute['startYear'] = pd.to_datetime(table_totale_brute['startYear'], format='%Y-%m-%d')
        # Filtrer le DataFrame en fonction de l'année choisie
        table_totale_brute = table_totale_brute[table_totale_brute['startYear'] == str(annee_choisie)]
    # -------------------------------------------------------------------------------------------------------------- DURÉE
    # Trier le DataFrame par la colonne 'runtime'
    table_totale_brute = table_totale_brute.sort_values(by='runtime')
    # Afficher les classes de durée disponibles
    classes_de_duree = ['0-60 min', '60-120 min', 'au-delà de 120 min']
    print("Classes de durée disponibles :", classes_de_duree)
    # Saisie manuelle de la classe de durée par l'utilisateur
    duree_classe = input("Choisissez une classe de durée (laissez vide pour tous) : ")
    if duree_classe:
        # Filtrer le DataFrame en fonction de la classe de durée saisie
        if duree_classe == '0-60 min':
            table_totale_brute = table_totale_brute[table_totale_brute['runtime'] <= 60]
        elif duree_classe == '60-120 min':
            table_totale_brute = table_totale_brute[(table_totale_brute['runtime'] > 60) & (table_totale_brute['runtime'] <= 120)]
        elif duree_classe == 'au-delà de 120 min':
            table_totale_brute = table_totale_brute[table_totale_brute['runtime'] > 120]
    # -------------------------------------------------------------------------------------------------------------- ACTEUR
    # Choix de l'acteur
    nom_acteur_choisi = input("Choisissez un nom d'acteur (laissez vide pour tous) : ").lower()
    if nom_acteur_choisi:
        # Filtrer les données pour l'acteur choisi dans la colonne 'cast'
        table_totale_brute = table_totale_brute[table_totale_brute['cast'].str.lower().str.contains(nom_acteur_choisi, na=False)]
    # -------------------------------------------------------------------------------------------------------------- RÉALISATEUR
    # Choix du réalisateur
    realisateur_choisi = input("Choisissez un réalisateur (laissez vide pour tous) : ").lower()
    if realisateur_choisi:
        # Filtrer les données pour le réalisateur choisi dans la colonne 'directors'
        table_totale_brute = table_totale_brute[table_totale_brute['Director'].str.lower().str.contains(realisateur_choisi, na=False)]
    # NOTE
    note_choisie = input("Choisissez une note (laissez vide pour tous) : ")  # faire des classes de valeurs
    if note_choisie:
        # la colonne 'vote_average' a le même type que la valeur entrée
        table_totale_brute['vote_average'] = table_totale_brute['vote_average'].astype(float)
        # Filtrer les données pour la note choisie
        table_totale_brute = table_totale_brute[table_totale_brute['vote_average'] == float(note_choisie)]
    # LANGUE
    # Saisie manuelle de la langue par l'utilisateur
    langue_choisie = input(f"Choisissez une langue entre (laissez vide pour tous) :   ")
    if langue_choisie:
        table_totale_brute = table_totale_brute.loc[table_totale_brute['original_language'] == langue_choisie]
    return table_totale_brute
# Utilisation de la fonction
resultat = filtrer_table(table_totale_brute)
# Afficher le DataFrame résultant
display(resultat)

Classes de durée disponibles : ['0-60 min', '60-120 min', 'au-delà de 120 min']


KeyError: 'director'

In [4]:
langue = sorted(table_totale_brute['original_language'].unique())
print(langue)

['ab', 'afrikaans', 'ak', 'allemand', 'amharique', 'anglais', 'arabe', 'assamais', 'ay', 'azéri', 'ba', 'basque', 'bengali', 'biélorusse', 'bm', 'bo', 'bs', 'bulgare', 'ca', 'ce', 'chinois mandarin', 'cn', 'coréen', 'croate', 'cs', 'da', 'dv', 'dz', 'eo', 'espagnol', 'et', 'ff', 'finnois', 'fo', 'français', 'fy', 'ga', 'gallois', 'gd', 'gl', 'gn', 'grec', 'gujarati', 'géorgien', 'hausa', 'he', 'hindi', 'ht', 'hu', 'hy', 'ig', 'indonésien', 'is', 'italien', 'iu', 'japonais', 'jv', 'kannada', 'khmer', 'kk', 'kl', 'ks', 'kurde', 'ky', 'latin', 'lb', 'lg', 'li', 'ln', 'lo', 'lt', 'lv', 'macédonien', 'malais', 'malayalam', 'marathi', 'mh', 'mi', 'mn', 'mo', 'mt', 'my', 'nb', 'novérgien', 'ny', 'néerlandais', 'népalais', 'oc', 'oriya', 'os', 'ourdou', 'persan', 'polonais', 'portugais', 'ps', 'punjabi', 'qu', 'rm', 'roumain', 'russe', 'rw', 'sa', 'sc', 'se', 'serbe', 'sh', 'si', 'sk', 'sl', 'sm', 'somali', 'sq', 'st', 'su', 'sv', 'swahili', 'tamoul', 'telugu', 'tg', 'thaï', 'tk', 'tl', 'tn', 